In [5]:
import sys
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
from lib.utils import *
from lib.plot import get_optimization_solution
import itertools
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
config_filename = "../data/YAML/region_y3_div.yaml"
n_iter_UE = 250
# percNonUVal = 80
n_time = 36
n_time_inc_start = 13
n_time_inc_end = 24
step_size_UE = 0.01
region_ = "region_y3_div"
setting_region = "6_9_AVG5_th1_pad_MultipleTimes"
fileName0 = "_StartHour_6_AVG5_pad_theta1e+00"
folderName0 = "Mar2May_2018_new_5-22_"
fairness = "0_0_0_100_0" # Corresponds to RHSMultiplier = [1, 1.1, 1.5, 2, 2.5]
n_iter_ADMM = 15000
n_companies_ADMM = 1
n_companies_cost_list = [1, 10, 100, 1000]
rho = 20
# rho_list = [20]
percNonUVal_list = [80, 85, 90, 95]
budget_list = [10000, 2000, 800, 200]
VOT_list = [2.63, 1.315]
solver_list = ["ADMM"] 
solve_binarization = 1 # 1=True, 0=False
seed_solver = 2
seed_ADMM = 2
seed_data = 2
MIPGap = 0.01

In [9]:
time_dict = {}
colsDict = {"solverName": [], "percNonUVal": [], "VOT": [], "budget": [], "nDeviated": []}
for solver_name, percNonUVal, VOT, budget in itertools.product(solver_list, percNonUVal_list, VOT_list, budget_list):
    seed_solving_algo = seed_ADMM if solver_name == "ADMM" else seed_solver
        
    # Get .mat file address 
    folderAddress = get_solution_result_folder_address(budget = budget, \
                                             seed_data = seed_data, 
                                             seed_solving_algo = seed_solving_algo,
                                             VOT = VOT,
                                             fairness = fairness,
                                             percNonUVal = percNonUVal,
                                             n_time_inc_start = n_time_inc_start,
                                             n_time_inc_end = n_time_inc_end,
                                             step_size = step_size_UE,
                                             iterRun = n_iter_UE,
                                             region_ = region_,
                                             setting_region = setting_region,
                                             solver_name = solver_name,
                                             n_iter_ADMM = n_iter_ADMM, 
                                             rho = rho,
                                            )
    fileName = get_solution_result_file_address(solver_name, MIPGap)
    fileAddress = os.path.join(folderAddress, fileName)
    # Load variables of solved optimization problem
    _, _, n_deviated = get_optimization_solution(fileAddress, solver_name)
    
    colsDict["solverName"].append(solver_name)
    colsDict["percNonUVal"].append(percNonUVal)
    colsDict["VOT"].append(VOT)
    colsDict["budget"].append(budget)
    colsDict["nDeviated"].append(n_deviated)

In [10]:
n_deviated_df = pd.DataFrame.from_dict(colsDict)

In [13]:
n_deviated_df[n_deviated_df['VOT']==2.63]

,solverName,percNonUVal,VOT,budget,nDeviated
0,ADMM,80,2.63,10000,195
1,ADMM,80,2.63,2000,123
2,ADMM,80,2.63,800,90
3,ADMM,80,2.63,200,49
8,ADMM,85,2.63,10000,151
9,ADMM,85,2.63,2000,101
10,ADMM,85,2.63,800,72
11,ADMM,85,2.63,200,42
16,ADMM,90,2.63,10000,94
17,ADMM,90,2.63,2000,74
